## Variation across windows

In [100]:
import pandas as pd 
from collections import Counter

demographic = "gender"
if demographic == "gender":
    df = pd.read_parquet('../../logits_results/joined/combined_gender_logits.parquet')
else:
    df = pd.read_parquet('../../logits_results/joined/combined_race_logits.parquet')

# keep only one model in model_name EleutherAI/pythia-70m-deduped	
df = df[df['model_name'] == 'EleutherAI/pythia-70m-deduped']
df = df[df['logit_type'] == 'hf_tf']
df = df[df['location_preprompt'] == 1]
df = df[df['template'] == 0]
df

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group
1,achilles tendinitis,male,-5.945312,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,119,10,179,66.480447,35.951834,gender
5,achilles tendinitis,female,-5.933594,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,58,10,179,32.402235,-36.590539,gender
9,achilles tendinitis,non-binary,-6.207031,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,2,10,179,1.117318,1017.318436,gender
13,achilles tendinitis,male,-5.945312,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,336,100,540,62.222222,27.243808,gender
17,achilles tendinitis,female,-5.933594,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,198,100,540,36.666667,-28.245271,gender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765883,visual anomalies,female,-5.710938,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,348,100,623,55.858748,9.312618,gender
1765887,visual anomalies,male,-5.312500,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,435,250,909,47.854785,-2.137453,gender
1765891,visual anomalies,female,-5.710938,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,474,250,909,52.145215,2.045430,gender
1765895,visual anomalies,male,-5.312500,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,153,50,394,38.832487,-20.587961,gender


In [101]:
# seprate to four df based on window size 10, 50, 100, 250
df_10 = df[df['window'] == '10']
df_50 = df[df['window'] == '50']
df_100 = df[df['window'] == '100']
df_250 = df[df['window'] == '250']

In [102]:
def get_ranking(df, count=3):
    disease = []
    rank = []
    # for every three rows
    for i in range(0, len(df), count):
        # get the disease
        disease.append(df.iloc[i]['disease'])
        # get the ranking of male, female and non-binary based on 'mention_count'
        subset = df.iloc[i:i+count]
        subset_sorted = subset.sort_values('mention_count', ascending=False)
        rank.append(list(subset_sorted['demographic']))

    return disease, rank

if demographic is 'gender':
    disease_10, rank_10 = get_ranking(df_10)
    disease_50, rank_50 = get_ranking(df_50)
    disease_100, rank_100 = get_ranking(df_100)
    disease_250, rank_250 = get_ranking(df_250)
else:
    disease_10, rank_10 = get_ranking(df_10, count=6)
    disease_50, rank_50 = get_ranking(df_50, count=6)
    disease_100, rank_100 = get_ranking(df_100, count=6)
    disease_250, rank_250 = get_ranking(df_250, count=6)


# should use disease window as 250 also because we can cover the most diseases
len(disease_10), len(disease_50), len(disease_100), len(disease_250)

<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_189215/1230011143.py:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if demographic is 'gender':


(78, 81, 81, 82)

In [98]:
# check the top 77 diseases in each window size's ranking whether they are the same
aggrement_count = 0
for i in range(80):
    if disease_10[i] == disease_50[i] == disease_100[i] == disease_250[i]:
        aggrement_count += 1
print(f'CACHED RUN for race: {aggrement_count} diseases are the same in the top 80 diseases in each window size\'s ranking')

CACHED RUN for race: 65 diseases are the same in the top 80 diseases in each window size's ranking


In [99]:
# check the top 77 diseases in each window size's ranking whether they are the same
aggrement_count = 0
for i in range(80):
    if disease_50[i] == disease_100[i] == disease_250[i]:
        aggrement_count += 1
print(f'CACHED RUN for race: {aggrement_count} diseases are the same in the top 80 diseases in each window size\'s ranking')

CACHED RUN for race: 80 diseases are the same in the top 80 diseases in each window size's ranking


In [103]:
# check the top 77 diseases in each window size's ranking whether they are the same
aggrement_count = 0
for i in range(77):
    if disease_10[i] != disease_50[i] or disease_10[i] != disease_100[i] or disease_10[i] != disease_250[i]:
        aggrement_count += 1
print(f'CACHED RUN for gender: {aggrement_count} diseases are the same in the top 77 diseases in each window size\'s ranking')

CACHED RUN for gender: 73 diseases are the same in the top 77 diseases in each window size's ranking
